# Optional lab: Simple neural network

In this lab, we will build a small neural network using Numpy

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))
    
def load_coffee_data():
    rng = np.random.default_rng(2)
    X = rng.random(400).reshape(-1,2)
    X[:,1] = X[:,1] * 4 + 11.5          # 12-15 min is best
    X[:,0] = X[:,0] * (285-150) + 150  # 350-500 F (175-260 C) is best
    Y = np.zeros(len(X))
    
    i=0
    for t,d in X:
        y = -3/(260-175)*t + 21
        if (t > 175 and t < 260 and d > 12 and d < 15 and d<=y ):
            Y[i] = 1
        else:
            Y[i] = 0
        i += 1

    return (X, Y.reshape(-1,1))

X, y = load_coffee_data()
norm_l = tf.keras.layers.Normalization(axis=-1)
norm_l.adapt(X)

You will define the my_dense() function which computes the activations of a dense layer.

In [5]:
def my_dense(a_in, W, b):
    units = W.shape[1]
    a_out = np.zeros(units)
    for i in range(units):
        z = np.dot(W[:, i], a_in) + b[i];
        a_out[i] = sigmoid(z)
    return a_out

The following cell builds a two-layer neural network utilizing the `my_dense` subroutine above.

In [7]:
def my_sequential(x, W1, b1, W2, b2):
    a1 = my_dense(x, W1, b1)
    a2 = my_dense(a1, W2, b2)
    return a2

In [8]:
W1_tmp = np.array( [[-8.93,  0.29, 12.9 ], [-0.1,  -7.32, 10.81]] )
b1_tmp = np.array( [-9.82, -9.28,  0.96] )
W2_tmp = np.array( [[-31.18], [-27.59], [-32.56]] )
b2_tmp = np.array( [15.41] )

## Predictions
Let's start by writing a routine similar to Tensorflow's model.predict(). This will take a matrix  𝑋
  with all  𝑚
  examples in the rows and make a prediction by running the model.

In [10]:
def my_predict(X, W1, b1, W2, b2):
    m = X.shape[0]
    p = np.zeros((m, 1))
    for i in range(m):
        p[i, 0] = my_sequential(X[i], W1, b1, W2, b2)
    return p

In [11]:
X_tst = np.array([
    [200,13.9],  # postive example
    [200,17]])   # negative example
X_tstn = norm_l(X_tst)  # remember to normalize
predictions = my_predict(X_tstn, W1_tmp, b1_tmp, W2_tmp, b2_tmp)
print(predictions)

tf.Tensor([-0.46714395  0.4159052 ], shape=(2,), dtype=float32)
tf.Tensor([-0.46714395  3.1631167 ], shape=(2,), dtype=float32)
[[9.71932288e-01]
 [3.28978626e-08]]


C:\Users\Windows\AppData\Local\Temp\ipykernel_14636\1837212533.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  p[i, 0] = my_sequential(X[i], W1, b1, W2, b2)


To convert the probabilities to a decision, we apply a threshold:

In [13]:
yhat = np.zeros_like(predictions)
for i in range(len(predictions)):
    if predictions[i] >= 0.5:
        yhat[i] = 1
    else:
        yhat[i] = 0
print(f"decisions = \n{yhat}")

decisions = 
[[1.]
 [0.]]
